In [1]:
import torch
import math
import os


if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)} is available.")
else:
    print("No GPU available. Training will run on CPU.")

verbose = False

GPU: NVIDIA RTX A6000 is available.


## MeMo Tokenizer and input

In [2]:
from MeMoPyTorch.modelling_memo_tokenizer import MeMoTokenizer

In [3]:
max_length = 12 
tokenizer = MeMoTokenizer.from_pretrained("EleutherAI/gpt-neox-20b", 
                                          truncation_side = 'left',
                                          padding_side='left', max_length=max_length, head_number=4)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPTNeoXTokenizer'. 
The class this function is called from is 'MeMoTokenizer'.


Setting pad token and pad token id = <|endoftext|>, 0


In [4]:
with open("testo_di_prova.txt") as my_first_text_f:
    my_first_text = my_first_text_f.read()

token_ids = tokenizer.encode(my_first_text)#, return_tensors='pt')
print(token_ids) # return max len + 1 

(tensor([[18886,   256, 36144,  4164,  1809,    80,  1448,   295,   532,  1584,
            13, 50190]]), tensor([[  256, 36144,  4164,  1809,    80,  1448,   295,   532,  1584,    13,
         50190,    15]]))


In [5]:
memo_input = tokenizer.get_text_batch_encoding([my_first_text, my_first_text[0:10]])
memo_input.keys()

dict_keys([0, 1, 2, 3])

In [6]:
memo_input[0]['input_ids'].shape

torch.Size([52, 12])

## MeMo Embedding layer

In [7]:
from MeMoPyTorch.modelling_memo_embedding import MeMoEmbedding

In [8]:
d,h,l = 1024, 4, 3

In [9]:
embedding = MeMoEmbedding(
    num_embeddings=tokenizer.vocab_size,
    embedding_dim=d,
    padding_idx=tokenizer.pad_token_id, #0
    _freeze=True
)

MeMo embedding initilialization


In [10]:
input_tokens_ids = tokenizer(['Test', 'Un altro Test'])['input_ids']
print(input_tokens_ids)

input_embeddings = embedding.forward(input_tokens_ids)
input_embeddings

tensor([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
         5089],
        [   0,    0,    0,    0,    0,    0,    0,    0,    0, 2447, 6945,  287,
         6004]])


tensor([[[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         ...,
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [-0.0161,  0.0099, -0.0306,  ..., -0.0136,  0.0123, -0.0044]],

        [[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         ...,
         [ 0.0047,  0.0229,  0.0024,  ..., -0.0326,  0.0234, -0.0191],
         [-0.0210, -0.0099,  0.0065,  ...,  0.0447, -0.0401,  0.0088],
         [-0.0516,  0.0538,  0.0633,  ..., -0.0263,  0.0363, -0.0175]]])

In [11]:
memo_input = tokenizer.get_text_batch_encoding([my_first_text, my_first_text[10:30]])

memo_input[0]['input_ids'].shape

torch.Size([52, 12])

In [12]:
memo_input[0]['input_ids'][0:10]

tensor([[    0,     0,     0,     0,     0,     0,     0,     0,     0, 38577,
         17622,  1073],
        [  372,     8,  9718,    74,   843,   936,  4164, 43876, 41380,   258,
           367,   727],
        [ 5507,   313, 15723,   445,  2721,    13,  3435,  3414,   358,  3381,
         15410,    26],
        [ 9776,  1266,    74,    13,   337, 11703,   639, 39337,  1638,  1540,
            10, 12187],
        [  440,  2314,  4173,   299,  8913,  2942,   250,   352,  6770,    80,
            13,  2248],
        [  861,   410,   372, 32924,  1073, 33813,   445,  2721,   299,  2248,
            80,  1484],
        [ 1073,   659,  4611,  1073,   391,   300,   466,  5711, 14804,  1431,
           304, 19702],
        [   74,    15, 14929,  1327,  1323, 10081, 24843, 15438,   412, 16406,
         38055,  9821],
        [ 3737,  1073,   391,   300,   466,  5711, 39814,   260,   770,  5991,
           313,  1962],
        [18006, 22217, 42722, 10863,   262,  7958,  1593, 12704,  5940,  

In [13]:
input_embeddings = embedding.encode(memo_input[0]['input_ids'])
output_symbols = embedding.encode(memo_input[0]['labels'])

input_embeddings.shape, output_symbols.shape

(torch.Size([52, 12, 1024]), torch.Size([52, 12, 1024]))

In [14]:
decoded, _ = embedding.decode(input_embeddings)
print(decoded.shape)

decoded[0:10]

torch.Size([52, 12])


tensor([[    0,     0,     0,     0,     0,     0,     0,     0,     0, 38577,
         17622,  1073],
        [  372,     8,  9718,    74,   843,   936,  4164, 43876, 41380,   258,
           367,   727],
        [ 5507,   313, 15723,   445,  2721,    13,  3435,  3414,   358,  3381,
         15410,    26],
        [ 9776,  1266,    74,    13,   337, 11703,   639, 39337,  1638,  1540,
            10, 12187],
        [  440,  2314,  4173,   299,  8913,  2942,   250,   352,  6770,    80,
            13,  2248],
        [  861,   410,   372, 32924,  1073, 33813,   445,  2721,   299,  2248,
            80,  1484],
        [ 1073,   659,  4611,  1073,   391,   300,   466,  5711, 14804,  1431,
           304, 19702],
        [   74,    15, 14929,  1327,  1323, 10081, 24843, 15438,   412, 16406,
         38055,  9821],
        [ 3737,  1073,   391,   300,   466,  5711, 39814,   260,   770,  5991,
           313,  1962],
        [18006, 22217, 42722, 10863,   262,  7958,  1593, 12704,  5940,  

In [15]:
decoded[0:10] == memo_input[0]['input_ids'][:10]

tensor([[True, True, True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True, True, True]])

In [16]:
sims = embedding.weight @ embedding.weight.T
display(sims)
diag_sum = torch.sum(sims[1:, 1: ].diag()) # almost 1 in each entry
print(diag_sum) # obs vs expected
print(torch.sum(sims[1:, 1:]) - diag_sum) #almost 0... more or less

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.9347, -0.0361,  ...,  0.0141, -0.0392, -0.0228],
        [ 0.0000, -0.0361,  1.0207,  ...,  0.0225,  0.0086, -0.0225],
        ...,
        [ 0.0000,  0.0141,  0.0225,  ...,  1.0130, -0.0314, -0.0594],
        [ 0.0000, -0.0392,  0.0086,  ..., -0.0314,  0.9786, -0.0431],
        [ 0.0000, -0.0228, -0.0225,  ..., -0.0594, -0.0431,  0.9813]])

tensor(50260.8867)
tensor(-2720.1641)


## Test layer and MeMo CMM

In [17]:
from MeMoPyTorch.modelling_memo_layer import MeMoLayer, ProjectionSequence, ProjectionTokens, CorrelationMatrixMemory

### Check initialization of each matrix

In [18]:
d,h,l = 1024, 4, 3
proj = ProjectionSequence(d, d*h)
print(proj.weight.shape, proj.extra_repr())
(proj.weight.T @ proj.weight).diag(), (proj.weight @ proj.weight.T).diag()

torch.Size([4096, 1024]) (trasposed wrt saved one) in_features=4096, out_features=1024


(tensor([0.9743, 0.9976, 0.9968,  ..., 0.9491, 1.0334, 1.0173],
        grad_fn=<DiagonalBackward0_copy>),
 tensor([0.2565, 0.2527, 0.2624,  ..., 0.2417, 0.2660, 0.2444],
        grad_fn=<DiagonalBackward0_copy>))

In [19]:
Prj = torch.normal(0, 1/math.sqrt(d*h), size=(d,d*h))
Prj = torch.transpose(Prj, 0, 1)
(Prj.T @ Prj).diag(), (Prj @ Prj.T).diag()

(tensor([1.0152, 0.9706, 0.9694,  ..., 0.9638, 0.9894, 1.0372]),
 tensor([0.2450, 0.2314, 0.2501,  ..., 0.2391, 0.2572, 0.2516]))

In [20]:
print(d)
print(h)

d_k = d // h

W_v = ProjectionTokens(d, d_k)

print(W_v.weight.shape, W_v.extra_repr())


### always used transposed! so check with .T
(W_v.weight.T @ W_v.weight).diag(), (W_v.weight @ W_v.weight.T).diag()

1024
4
torch.Size([256, 1024]) in_features=1024, out_features=256


(tensor([0.9536, 0.9282, 0.8832,  ..., 0.9966, 0.8963, 0.9501],
        grad_fn=<DiagonalBackward0_copy>),
 tensor([3.9666, 4.0939, 4.2988, 4.0052, 4.1969, 4.2777, 4.1574, 4.1704, 3.7788,
         4.3970, 3.9970, 4.1488, 3.8178, 3.9645, 3.8420, 4.0947, 3.9161, 4.2043,
         3.6926, 4.0570, 4.1142, 3.7652, 4.0730, 3.8482, 4.0229, 3.8988, 3.7216,
         4.0209, 4.0156, 3.9800, 4.0440, 4.2623, 4.0462, 3.8154, 3.7813, 4.0432,
         3.8823, 3.8828, 3.8986, 3.9567, 3.8791, 4.2087, 4.1604, 3.9781, 3.9351,
         4.1483, 3.8459, 3.9146, 3.9400, 4.1593, 3.9039, 3.9640, 4.0969, 3.7689,
         4.1690, 3.8867, 4.3060, 4.0505, 3.9825, 3.9963, 3.9738, 4.0899, 3.9403,
         3.8676, 4.1243, 4.3099, 4.0107, 3.7395, 3.7215, 3.9500, 3.7980, 3.9023,
         3.9330, 4.0665, 4.0252, 4.1403, 4.0016, 3.8523, 3.7321, 3.9791, 4.4043,
         3.8752, 3.9721, 4.0179, 3.8165, 3.8789, 4.0939, 4.0955, 3.8693, 4.2991,
         4.1217, 4.0396, 3.6650, 4.0047, 4.0859, 4.0621, 4.0245, 4.0620, 3.9047,
  

In [21]:
W_v_single_head = torch.normal(0, 1/math.sqrt(d_k), size=(d,d_k))

(W_v_single_head.T @ W_v_single_head).diag(), (W_v_single_head @ W_v_single_head.T).diag()

(tensor([3.9115, 3.8792, 4.3531, 3.9076, 4.0819, 3.9219, 3.7893, 3.8794, 4.2437,
         3.7041, 4.3074, 4.0168, 4.0283, 3.7861, 3.9509, 4.0091, 3.9561, 3.9187,
         3.8834, 3.7614, 3.8834, 3.7877, 3.7784, 3.9032, 4.0129, 4.1955, 4.0918,
         3.9693, 4.1765, 4.0418, 3.9616, 3.9822, 3.9425, 4.1296, 3.8702, 3.8956,
         4.0325, 4.0417, 3.8514, 3.7178, 3.9856, 4.0762, 4.0763, 4.2492, 4.1772,
         4.0986, 4.0013, 3.9240, 4.1282, 3.9898, 4.3139, 4.0509, 3.7595, 4.2079,
         3.8705, 4.3164, 3.9169, 3.5837, 4.1498, 3.8917, 4.0558, 3.8311, 3.8746,
         4.3504, 4.1333, 4.0077, 4.1311, 3.9522, 4.0267, 4.0657, 4.3141, 4.3397,
         3.8193, 3.7877, 4.0753, 4.1815, 4.0575, 3.9062, 3.8312, 4.1112, 4.0141,
         3.7511, 4.1266, 3.6544, 4.3672, 4.0148, 4.0176, 3.8992, 3.9791, 3.9479,
         3.9572, 4.0422, 3.8299, 4.1287, 4.1621, 4.1473, 4.3931, 3.9538, 4.2009,
         3.7590, 3.9368, 4.1343, 3.8517, 4.2789, 4.0345, 3.6222, 4.1233, 3.8122,
         3.9075, 4.0444, 3.9

### Check memorization on single layer

In [22]:
d,h,l = 1024, 4, 3

layer = MeMoLayer(d, h)
layer

MeMoLayer(
  (W_v_single_head): ProjectionTokens(in_features=1024, out_features=256)
  (Prj): ProjectionSequence((trasposed wrt saved one) in_features=4096, out_features=1024)
  (CMM): CorrelationMatrixMemory(in_features=1024, out_features=1024)
)

In [23]:
batch_size, current_length, d = input_embeddings.shape
batch_size, current_length, d 

(52, 12, 1024)

In [24]:
output_symbols.shape

torch.Size([52, 12, 1024])

In [25]:
current_length = int(input_embeddings.shape[1]/ h)

input_sequence = input_embeddings.reshape((batch_size, current_length, h, d))

current_output_symbols = output_symbols[:, [(x+1)*h-1 for x in range(0,current_length)]]
j = 2 
print(sum(sum(input_sequence[0][j] == input_embeddings[0][4*j:4*(j+1)])), input_sequence[0][j].shape)

(batch_size, blocks,h,d) = input_sequence.shape

tensor(4096) torch.Size([4, 1024])


In [26]:
input_sequence.shape, current_output_symbols.shape

(torch.Size([52, 3, 4, 1024]), torch.Size([52, 3, 1024]))

In [27]:
layer = MeMoLayer(d, h)
display(layer)
## update the input sequence for the next layer
_, seq_encoding_for_the_last_layer = layer.memorize(input_sequence, current_output_symbols, is_last=False)
layer.directly_memorize(seq_encoding_for_the_last_layer)

MeMoLayer(
  (W_v_single_head): ProjectionTokens(in_features=1024, out_features=256)
  (Prj): ProjectionSequence((trasposed wrt saved one) in_features=4096, out_features=1024)
  (CMM): CorrelationMatrixMemory(in_features=1024, out_features=1024)
)

In [28]:
input_sequence.shape, input_sequence[3].shape # batch (52 elements of chunks 4*4*1024)

(torch.Size([52, 3, 4, 1024]), torch.Size([3, 4, 1024]))

In [29]:
_, seq_encoding_for_the_last_layer = layer.retrieve(input_sequence)

print(seq_encoding_for_the_last_layer.shape)

torch.Size([52, 1024])


In [30]:
logits = layer.directly_retrieve(seq_encoding_for_the_last_layer)

In [31]:
retreived_output_symbol_vector, m = embedding.decode(logits)
print(retreived_output_symbol_vector, m)

tensor([48505, 20110,  1108, 48019,    80, 19216,  9718,  1113,  4927,    66,
        19216,  1448,  2122, 41530,   187,  4172,   246,   659, 10986, 30975,
           80, 12931,   352, 14134,  2721,   258,  8830,    87,   826,    15,
        17532,   729, 26798, 41070,  6575,   299,   266,  3737, 20889,   287,
          512,   354,   250,   247,    70,   275, 16128,  2680,    74, 13679,
           15,   209]) tensor([0.6741, 0.9087, 0.9506, 1.0715, 1.0061, 1.3768, 1.0262, 1.0275, 1.0287,
        1.6682, 1.2615, 0.9422, 1.1550, 0.9810, 0.9824, 1.0470, 1.0436, 0.9405,
        1.0863, 1.0681, 0.9732, 0.9020, 0.9914, 0.9732, 1.3128, 0.9585, 0.9333,
        0.9433, 0.8790, 1.1479, 0.9481, 1.0695, 0.9663, 0.9757, 1.0728, 0.8427,
        0.9790, 1.0694, 1.0527, 1.0305, 0.9935, 0.9955, 0.9932, 1.0478, 0.9997,
        0.8600, 0.8966, 0.9301, 0.9483, 0.8895, 1.1181, 1.0606],
       grad_fn=<MaxBackward0>)


In [32]:
o = embedding.decode(current_output_symbols[:, -1])[0]
display(o)

print(sum(o == retreived_output_symbol_vector), 'over', retreived_output_symbol_vector.shape)

tensor([48505, 20110,  1108, 48019,    80, 19216,  9718,  1113,  4927,    66,
        19216,  1448,  2122, 41530,   187,  4172,   246,   659, 10986, 30975,
           80, 12931,   352, 14134,  2721,   258,  8830,    87,   826,    15,
        17532,   729, 26798, 41070,  6575,   299,   266,  3737, 20889,   287,
          512,   354,   250,   247,    70,   275, 16128,  2680,    74, 13679,
           15,   209])

tensor(52) over torch.Size([52])


In [33]:
#### test single block

In [34]:
print(input_sequence[3].shape, current_output_symbols[3].shape)
print(input_sequence[3][0].shape, current_output_symbols[3][0].shape)

torch.Size([3, 4, 1024]) torch.Size([3, 1024])
torch.Size([4, 1024]) torch.Size([1024])


In [35]:
total = 0
correct = 0 

for batch_index in range(len(memo_input[0]['input_ids'])):
    #print("input ids", memo_input[0]['input_ids'][batch_index])
    #print()
    
    for i in range(len(current_output_symbols[batch_index])):
        #display(embedding.decode(input_sequence[batch_index][i]), embedding.decode(current_output_symbols[batch_index][i]))
        true = embedding.decode(current_output_symbols[batch_index][i])[0].item()
        
        _, seq_encoding_for_the_last_layer  = layer.retrieve(input_sequence[batch_index][i].unsqueeze(0).unsqueeze(0))
        
        retreived_output_symbol_vector, m = embedding.decode(layer.directly_retrieve(seq_encoding_for_the_last_layer))
        pred = retreived_output_symbol_vector.item()

        total += 1
        correct += pred == true

print(f"{correct}/{total}")

153/156


### Check with batch size of 1 and output probs

In [36]:
memo_input = tokenizer.get_text_batch_encoding(['this is a test for a very short short sequence of 12 tokens'])[0]
input_ids, labels = memo_input['input_ids'], memo_input['labels']
print(input_ids, labels)

input_embeddings = embedding.encode(input_ids)
#print(input_embeddings.shape)

output_embeddings = embedding.encode(labels)
#print(output_embeddings.shape)


current_length = max_length

current_length = int(current_length/h)
input_sequence = input_embeddings.reshape((1, current_length, h, d))

output_symbols = output_embeddings[:, [(x+1)*h-1 for x in range(0,current_length)]] ## the output symbol is always the same tokem?
print(embedding.decode(output_symbols))
input_sequence.shape, output_symbols.shape

tensor([[2520,  310,  247, 1071,  323,  247, 1077, 2159, 2159, 3425,  273, 1249]]) tensor([[  310,   247,  1071,   323,   247,  1077,  2159,  2159,  3425,   273,
          1249, 21761]])
(tensor([[  323,  2159, 21761]]), tensor([[0.9441, 0.9547, 1.0388]]))


(torch.Size([1, 3, 4, 1024]), torch.Size([1, 3, 1024]))

In [37]:
embedding.decode(input_sequence)[0], embedding.decode(output_symbols)[0], input_ids

(tensor([[[2520,  310,  247, 1071],
          [ 323,  247, 1077, 2159],
          [2159, 3425,  273, 1249]]]),
 tensor([[  323,  2159, 21761]]),
 tensor([[2520,  310,  247, 1071,  323,  247, 1077, 2159, 2159, 3425,  273, 1249]]))

In [38]:
input_sequence.shape

torch.Size([1, 3, 4, 1024])

In [39]:
output_symbols.shape

torch.Size([1, 3, 1024])

In [40]:
layer = MeMoLayer(d, h)
display(layer)

## update the input sequence for the next layer
_, seq_encoding_for_the_last_layer = layer.memorize(input_sequence, output_symbols, is_last=False)
layer.directly_memorize(seq_encoding_for_the_last_layer)

for i in range(0,3):
    _, seq_encoding_for_the_last_layer = layer.retrieve(input_sequence[0][i].unsqueeze(0).unsqueeze(0))
    print(seq_encoding_for_the_last_layer.shape)
                                                        
    retreived_output_symbol_vector, m = embedding.decode(layer.directly_retrieve(seq_encoding_for_the_last_layer))
    print(retreived_output_symbol_vector, m)

MeMoLayer(
  (W_v_single_head): ProjectionTokens(in_features=1024, out_features=256)
  (Prj): ProjectionSequence((trasposed wrt saved one) in_features=4096, out_features=1024)
  (CMM): CorrelationMatrixMemory(in_features=1024, out_features=1024)
)

torch.Size([1, 1024])
tensor([323]) tensor([0.9780], grad_fn=<MaxBackward0>)
torch.Size([1, 1024])
tensor([2159]) tensor([1.0270], grad_fn=<MaxBackward0>)
torch.Size([1, 1024])
tensor([21761]) tensor([1.1140], grad_fn=<MaxBackward0>)


## Test the entire MeMo model

In [41]:
from MeMoPyTorch.modelling_memo import MeMo

In [42]:
from MeMoPyTorch.modelling_memo_tokenizer import MeMoTokenizer

In [43]:
with open("testo_di_prova.txt") as my_first_text_f:
    my_first_text = my_first_text_f.read()

In [44]:
max_length = 384
print(max_length, h)
tokenizer = MeMoTokenizer.from_pretrained("EleutherAI/gpt-neox-20b", 
                                          padding_side='left', truncation_side='left', 
                                          max_length=max_length, head_number=h)

384 4


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPTNeoXTokenizer'. 
The class this function is called from is 'MeMoTokenizer'.


Setting pad token and pad token id = <|endoftext|>, 0


In [45]:
memo_input = tokenizer.memo_heads_encode(my_first_text[0:10])
memo_input[0]['labels'].shape

torch.Size([1, 384])

In [46]:
device='cuda:0' #'cpu'

In [47]:
d,h,l

(1024, 4, 3)

In [48]:
model = MeMo(inner_dim=d, 
             num_of_heads=h, 
             num_of_layers=3, 
             chunk_length=max_length, 
             num_embeddings=tokenizer.vocab_size, 
             padding_idx=tokenizer.pad_token_id, 
             device=device) #MeMoModel
model

MeMo embedding initilialization


MeMo(
  (encoder): MeMoEmbedding(50254, 1024, padding_idx=0)
  (layers): ModuleList(
    (0-2): 3 x MeMoLayer(
      (W_v_single_head): ProjectionTokens(in_features=1024, out_features=256)
      (Prj): ProjectionSequence((trasposed wrt saved one) in_features=4096, out_features=1024)
      (CMM): CorrelationMatrixMemory(in_features=1024, out_features=1024)
    )
  )
)

In [49]:
import tqdm


class Evaluation:
    def check_memorization(self, model, tokenizer, text, # device='cpu',
                           starting_point=None):
        if starting_point == None:
            basic_block = model.h ** model.l
        else:
            basic_block = starting_point
        
        
        input_ = tokenizer(my_first_text, padding='longest', truncation='do_not_truncate', max_length=None)
        input_ = tokenizer.pad(input_, pad_to_multiple_of=basic_block)
        input_ids = input_['input_ids']
                
        count = 0
        correct = 0
        max_length = tokenizer.max_length
        (batch_size, number_of_tokens) = input_ids.shape

        #print(f"(batch_size, number_of_tokens) = {(batch_size, number_of_tokens)}")
        
        for i in tqdm.tqdm(range(basic_block,  number_of_tokens - 1)):
            text_tokens = input_ids[:, i - basic_block:i]
            
            (batch_size, number_of_tokens) = text_tokens.shape
            
            text_tokens = torch.concat((torch.zeros((batch_size, max_length-1-number_of_tokens), 
                                                    dtype=torch.int), 
                                        text_tokens), axis=1
                                      )
            
            #print(i - basic_block, i)
            out, max_value = model.retrieve(text_tokens)
            #print(out, input_ids[:, i])
            #print(out[0].item())
            
            count += batch_size
            correct += torch.sum(out.to('cpu') == input_ids[:, i])
        
                           
        return correct / count

    def check_pretokenized(self, model, tokenizer, input_ids,# device='cpu',
                           starting_point=None):
        if starting_point == None:
            basic_block = model.h ** model.l
        else:
            basic_block = starting_point
                
        count = 0
        correct = 0
        max_length = tokenizer.max_length
        (batch_size, number_of_tokens) = input_ids.shape

        #print(f"(batch_size, number_of_tokens) = {(batch_size, number_of_tokens)}")
        
        for i in tqdm.tqdm(range(basic_block,  number_of_tokens - 1)):
            text_tokens = input_ids[:, i - basic_block:i]
            
            (batch_size, number_of_tokens) = text_tokens.shape
            
            text_tokens = torch.concat((torch.zeros((batch_size, max_length-1-number_of_tokens), 
                                                    dtype=torch.int), 
                                        text_tokens), axis=1
                                      )
            
            #print(i - basic_block, i)
            out, max_value = model.retrieve(text_tokens)
            #print(out, input_ids[:, i])
            #print(out[0].item())
            
            count += batch_size
            correct += torch.sum(out.to('cpu') == input_ids[:, i])
        
                           
        return correct / count
        

In [50]:
model = MeMo(inner_dim=d, 
             num_of_heads=h, 
             num_of_layers=l, 
             chunk_length=max_length, 
             num_embeddings=tokenizer.vocab_size, 
             padding_idx=tokenizer.pad_token_id, 
             device=device)

memo_input = tokenizer.get_text_batch_encoding([my_first_text]*8)
model.memorize_text(memo_input)

e = Evaluation()
out = e.check_pretokenized(model, tokenizer, memo_input[0]['input_ids'])
print("Degree of memorization: %f ", out)

MeMo embedding initilialization


100%|██████████| 319/319 [00:01<00:00, 200.71it/s]

Degree of memorization: %f  tensor(0.9091)


In [51]:
model = MeMo(inner_dim=d, 
             num_of_heads=h, 
             num_of_layers=l, 
             chunk_length=max_length, 
             num_embeddings=tokenizer.vocab_size, 
             padding_idx=tokenizer.pad_token_id, 
             device=device)
print("CMM pre learning")
display(model.layers[0].CMM.weight)



memo_input = tokenizer.get_text_batch_encoding([my_first_text]*8)
(bs, ml) = memo_input[0]['input_ids'].shape

for b in range(bs):
    memo_single_input = {h: {k: memo_input[h][k][b].unsqueeze(0) for k in memo_input[h]} for h in memo_input}
    #print(memo_single_input[0]['input_ids'].shape)

    model.memorize_text(memo_single_input)

e = Evaluation()
out = e.check_pretokenized(model, tokenizer, memo_input[0]['input_ids'])
print("Degree of memorization: %f ", out)

MeMo embedding initilialization
CMM pre learning


Parameter containing:
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0', requires_grad=True)

100%|██████████| 319/319 [00:01<00:00, 201.87it/s]

Degree of memorization: %f  tensor(0.8558)


In [52]:
Prj = model.layers[0].Prj.weight.detach().cpu()
CMM = model.layers[0].CMM.weight.detach().cpu()

Prj.T @ Prj

tensor([[ 1.0178e+00,  3.8739e-04,  1.7499e-02,  ..., -6.0468e-03,
          1.1178e-02, -9.1373e-03],
        [ 3.8739e-04,  1.0019e+00, -1.0599e-03,  ..., -2.0136e-03,
          2.9460e-03, -2.7718e-02],
        [ 1.7499e-02, -1.0599e-03,  9.5958e-01,  ..., -1.0601e-02,
         -1.2269e-02,  7.5585e-03],
        ...,
        [-6.0468e-03, -2.0136e-03, -1.0601e-02,  ...,  9.8908e-01,
          6.3645e-03, -2.5074e-02],
        [ 1.1178e-02,  2.9460e-03, -1.2269e-02,  ...,  6.3645e-03,
          1.0045e+00, -7.4022e-03],
        [-9.1373e-03, -2.7718e-02,  7.5585e-03,  ..., -2.5074e-02,
         -7.4022e-03,  1.0261e+00]])

In [53]:
CMM

tensor([[ 0.0236, -0.0366, -0.0132,  ...,  0.0719,  0.0025, -0.0073],
        [ 0.0307,  0.0299,  0.0235,  ...,  0.0044,  0.0144, -0.0217],
        [ 0.0413, -0.0083,  0.0033,  ...,  0.0161,  0.0156, -0.0391],
        ...,
        [-0.0076,  0.0381,  0.0200,  ..., -0.0089,  0.0288, -0.0411],
        [-0.0190, -0.0297, -0.0008,  ...,  0.0185, -0.0133, -0.0069],
        [-0.0405, -0.0487, -0.0048,  ...,  0.0089, -0.0061,  0.0177]])

In [54]:
out = e.check_pretokenized(model, tokenizer, memo_input[0]['input_ids'])
print("Degree of memorization: %f ", out)

100%|██████████| 319/319 [00:01<00:00, 199.52it/s]

Degree of memorization: %f  tensor(0.8558)


In [55]:
model.forget_text(memo_input)

In [56]:
out = e.check_pretokenized(model, tokenizer, memo_input[0]['input_ids'])
print("Degree of memorization: %f ", out)

100%|██████████| 319/319 [00:01<00:00, 201.88it/s]

Degree of memorization: %f  tensor(0.0878)
